In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/Users/anishkamukherjee/Documents/msg_spam/data/new_spam.csv")
df.head()

,result,text,num_chars,num_words,num_sents,text_trans
0,0,"go until jurong point, crazy.. available only ...",111,24,2,go jurong point crazi avail bugi n great world...
1,0,ok lar... joking wif u oni...,29,8,2,ok lar joke wif u oni
2,1,free entry in 2 a wkly comp to win fa cup fina...,155,37,2,free entri 2 wkli comp win fa cup final tkt 21...
3,0,u dun say so early hor... u c already then say...,49,13,1,u dun say earli hor u c alreadi say
4,0,"nah i don't think he goes to usf, he lives aro...",61,15,1,nah think goe usf live around though


In [3]:
df.shape

(5572, 6)

In [4]:
df.isna().sum()

result         0
text           0
num_chars      0
num_words      0
num_sents      0
text_trans    10
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
df.isna().sum()

result        0
text          0
num_chars     0
num_words     0
num_sents     0
text_trans    0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5562 entries, 0 to 5571
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   result      5562 non-null   int64 
 1   text        5562 non-null   object
 2   num_chars   5562 non-null   int64 
 3   num_words   5562 non-null   int64 
 4   num_sents   5562 non-null   int64 
 5   text_trans  5562 non-null   object
dtypes: int64(4), object(2)
memory usage: 304.2+ KB


## Model Building 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer()
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df["text_trans"]).toarray()

In [9]:
X.shape

(5562, 6707)

In [25]:
y = df["result"].values
y

array([0, 0, 1, ..., 0, 0, 0])

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [27]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score

gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [31]:
gnb.fit(X_train, y_train)
y_hat1 = gnb.predict(X_test)
print("The accuracy score is: ", accuracy_score(y_test, y_hat1) * 100)
print("The confusion matrix is: ", confusion_matrix(y_test, y_hat1))
print("The percision score is: ", precision_score(y_test, y_hat1) * 100)

The accuracy score is:  87.51123090745733
The confusion matrix is:  [[848 125]
 [ 14 126]]
The percision score is:  50.199203187250994


In [32]:
mnb.fit(X_train, y_train)
y_hat2 = mnb.predict(X_test)
print("The accuracy score is: ", accuracy_score(y_test, y_hat2) * 100)
print("The confusion matrix is: ", confusion_matrix(y_test, y_hat2))
print("The percision score is: ", precision_score(y_test, y_hat2) * 100)

The accuracy score is:  96.85534591194968
The confusion matrix is:  [[973   0]
 [ 35 105]]
The percision score is:  100.0


In [33]:
bnb.fit(X_train, y_train)
y_hat3 = bnb.predict(X_test)
print("The accuracy score is: ", accuracy_score(y_test, y_hat3) * 100)
print("The confusion matrix is: ", confusion_matrix(y_test, y_hat3))
print("The percision score is: ", precision_score(y_test, y_hat3) * 100)

The accuracy score is:  96.94519317160827
The confusion matrix is:  [[968   5]
 [ 29 111]]
The percision score is:  95.6896551724138


### The data is imbalanced so precision score matters the most in this case. So MultinomialNB seems to have the most accurate precision score.
- tfidf --> MNB

In [37]:
from sklearn.ensemble import VotingClassifier

vc = VotingClassifier(estimators=[('mnb', mnb), ('bnb', bnb)], voting='soft')

In [38]:
vc.fit(X_train, y_train)
y_hat4 = bnb.predict(X_test)
print("The accuracy score is: ", accuracy_score(y_test, y_hat4) * 100)
print("The confusion matrix is: ", confusion_matrix(y_test, y_hat4))
print("The percision score is: ", precision_score(y_test, y_hat4) * 100)

The accuracy score is:  96.94519317160827
The confusion matrix is:  [[968   5]
 [ 29 111]]
The percision score is:  95.6896551724138


In [40]:
import pickle

pickle.dump(tfidf, open("/Users/anishkamukherjee/Documents/msg_spam/pkls/vectorizer.pkl", "wb"))
pickle.dump(mnb, open("/Users/anishkamukherjee/Documents/msg_spam/pkls/model.pkl", "wb"))